In [1]:
from selenium import webdriver
import time
import pandas as pd 
import os
import csv
from bs4 import BeautifulSoup
from getpass import getpass
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By 
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# search_titles = ["business analyst","data analytics", "business intelligence","data science"]

# cities = ["Boston","New York","New Jersey"]
cities = ["Boston"]
search_titles = ["business analyst"]
# job_types = ["fulltime","parttime"]
job_types = ["fulltime"]

In [3]:
headers = ["Title","Desc"]
url1 = "https://www.indeed.com/jobs?q="
driver  = webdriver.Chrome(executable_path = "./chromedriver")

/var/folders/q2/rrpc1xwj3fd3w1qkrnk4q3j00000gn/T/ipykernel_47701/1501688740.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver  = webdriver.Chrome(executable_path = "./chromedriver")


In [4]:
rows = []
for stitle in search_titles:
    for city in cities:
        for type in job_types:
            path = f"https://www.indeed.com/jobs?q={stitle.replace(' ','+')}&l={city.replace(' ','+')}&sc=0kf%3Ajt({type})%3B"
            driver.implicitly_wait(1000)
            driver.get(path)
            driver.implicitly_wait(1000)
            print(path)
            time.sleep(2)
            jobs  = driver.find_elements(By.XPATH,"//div[@class = 'job_seen_beacon']")
            for item in jobs:
                temp = []
#                 time.sleep(2)
                index = jobs.index(item)
                timeout = time.time() + 5
                try:
                    item.click()
                    driver.implicitly_wait(10000000)
        #             time.sleep(3)
                    try:
                        job_url = item.find_elements(By.XPATH,"//a")
                        job_details = item.find_element(By.XPATH,"//div[@id = 'jobsearch-ViewjobPaneWrapper']")
                        if(job_details): 
                            desc = job_details.find_element(By.XPATH,"//div[@class = 'jobsearch-jobDescriptionText']").text                    
                            job_title = job_details.find_element(By.XPATH,"//h1[@class = 'icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title is-embedded']").text
                            temp.append(job_title)
                            company_name_selector = item.find_elements(By.XPATH,"//span[@class = 'companyName']")
                            company_rating = item.find_elements(By.XPATH,"//span[@class = 'ratingsDisplay withRatingLink']")
                            posted_date = item.find_elements(By.XPATH,"//span[@class = 'date']")
#                             if(len(company_name_selector)>index):
#                                 name = company_name_selector[index].text
#                                 temp.append(name)
#                             else:
#                                 temp.append("Nan")
                            if(len(job_url)>index):
                                url = job_url[index].get_attribute('href')
                                temp.append(url)
                            else:
                                temp.append("Nan")
                            if(len(job_url)>index):
                                url = job_url[index].get_attribute('href')
                                temp.append(url)
                            else:
                                temp.append("Nan")
                            if(len(company_rating)>index):
                                rating = company_rating[index].text
                                temp.append(rating)
                            else:
                                temp.append("Nan")
                            temp.append(city)
                            temp.append(type)
                            temp.append(desc)
                            if(len(posted_date)>index):
                                date = posted_date[index].text
                                if(len(date.split())>2):
                                    temp.append(date.split()[2])
                                else:
                                    temp.append("Nan")
                            else:
                                temp.append("Nan")
                           
                            rows.append(temp)
                    except Exception as e:
                        print(e)
                except StaleElementReferenceException:
                    continue

https://www.indeed.com/jobs?q=business+analyst&l=Boston&sc=0kf%3Ajt(fulltime)%3B


In [5]:
with open('indeed.csv', 'w', newline='', encoding='utf-8') as f:
                header = ['Job_Titile','Job_Url','Company_Url','Rating','Location','Job_type','Job_description','Posted_Date']
                writer = csv.writer(f)
                writer.writerow(header)
                writer.writerows(rows)